# TDSE - Heart Disease Risk Prediction: Logistic Regression Homework



In [2]:
%pip install numpy pandas matplotlib 

  Using cached numpy-2.4.1-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
  Using cached matplotlib-3.10.8-cp311-cp311-win_amd64.whl.metadata (52 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp311-cp311-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-12.1.0-cp311-cp311-win_amd64.whl.metadata (9.0 kB)
  Using cached pyparsing-3.3.2-py3-none-any.whl.metadata (5.8 kB)
Using cached numpy-2.4.1-cp311-cp311-win_amd64.whl (12.6 MB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.9 MB 6.7 MB/s eta 0:00:02
   --- ------------------------------------ 0.8/9.9 MB 6.7 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/9.9 MB 2.0 MB/s eta 0:00:

In [ ]:
import pandas as pd

df = pd.read_csv("Heart_Disease_Prediction.csv")


